# CODIGO DE EJECUCION

*NOTA: Para poder usar este código de ejecución hay que lanzarlo desde exactamente el mismo entorno en el que fue creado.*

*Se puede instalar ese entorno en la nueva máquina usando el environment.yml que creamos en el set up del proyecto*

*Copiar el proyecto1.yml al directorio y en el terminal o anaconda prompt ejecutar:*

conda env create --file proyecto1.yml --name proyecto1

In [2]:
import cloudpickle
import pandas as pd

ruta_proyecto = 'C:/Users/USER/Desktop/DSD4B (ZERO TO HERO)/03_MACHINE_LEARNING/08_CASOS/01_LEADSCORING'

nombre_fichero_datos = 'leads_2023.csv'

ruta_completa = ruta_proyecto + '/02_Datos/02_Validacion/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col='id',sep=';')
df = df.loc[(df.no_llamar != 'Otros') & (df.no_enviar_email != 'Yes') & (df.ult_actividad != 'Email Bounced')]


df_target= df['compra']
variables_finales = ['ambito',
                   'fuente',
                   'ocupacion',
                   'origen',
                   'paginas_vistas_visita',
                   'score_actividad',
                   'score_perfil',
                   'tiempo_en_site_total',
                   'ult_actividad',
                   'visitas_total'
                  ]
                     
df = df[variables_finales]


nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='rb') as file:
   pipe_ejecucion = cloudpickle.load(file)

scoring = pipe_ejecucion.predict_proba(df)[:, 1]
prediccion = pipe_ejecucion.predict(df)

In [3]:
scoring

array([0.8441944 , 0.0328453 , 0.7887266 , ..., 0.95205605, 0.3992923 ,
       0.8769303 ], dtype=float32)

In [4]:
df['prediction_prob']=scoring
df['prediction']=prediccion
df=df.reset_index()
df

,id,ambito,fuente,ocupacion,origen,paginas_vistas_visita,score_actividad,score_perfil,tiempo_en_site_total,ult_actividad,visitas_total,prediction_prob,prediction
0,660727,SG3 - Otros,Direct Traffic,Supervisor de almacen,Landing Page Submission,2.00,14.0,20.0,1532,Email Opened,2.0,0.844194,1
1,660719,SG1C - Aerolineas,Direct Traffic,Asistente de logistica,Landing Page Submission,1.00,13.0,17.0,305,Unreachable,1.0,0.032845,0
2,660681,SG1A - Agentes de Carga,Google,Asistente de logistica,Landing Page Submission,1.00,15.0,18.0,1428,Converted to Lead,2.0,0.788727,1
3,660664,NaN,Chat,NaN,API,0.00,15.0,15.0,0,Chat Conversation,0.0,0.022797,0
4,660624,SG2B - Exportadores,Direct Traffic,NaN,Landing Page Submission,2.00,14.0,14.0,71,Email Opened,2.0,0.073888,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2094,579753,SG2A - Importadores,Google,Asistente de logistica,Landing Page Submission,4.00,12.0,20.0,455,Form Submitted on Website,4.0,0.101377,0
2095,579728,SG2A - Importadores,Google,Asistente de logistica,Landing Page Submission,1.67,15.0,20.0,1283,Email Opened,5.0,0.947916,1
2096,579697,SG2B - Exportadores,Google,Asistente de logistica,Landing Page Submission,4.00,15.0,20.0,1016,Email Opened,8.0,0.952056,1
2097,579546,SG1C - Aerolineas,Direct Traffic,Asistente de logistica,Landing Page Submission,2.00,14.0,19.0,238,SMS Sent,2.0,0.399292,0


In [5]:
# Exportar el DataFrame a un archivo CSV
df.to_csv('prediccion_leads_2023.csv', index=False)

## Evaluamos la salida

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np

def calcular_metricas(real, scoring, umbral):
    
    roc_auc = roc_auc_score(real,scoring)
    
    #CALCULAR LA DECISION SEGUN EL UMBRAL
    predicho = np.where(scoring > umbral,1,0) 
    
    #CALCULAR TODAS LAS MÉTRICAS
    conf = confusion_matrix(real,predicho)

    tn, fp, fn, tp = conf.ravel()

    total_casos = df_target.shape[0]
    
    accuracy = (tn + tp) / total_casos
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    F1 = 2 * (precision * recall) / (precision + recall)

    #IMPRIMIR RESULTADOS
    print('\nMatriz de confusión\n',pd.DataFrame(conf))
    print('\nroc auc:',round(roc_auc,3))
    print('\naccuracy:',round(accuracy,3))
    print('\nprecision:',round(precision,3))
    print('\nrecall:',round(recall,3))
    print('\nF1:',round(F1,3))

In [7]:
calcular_metricas(df_target, scoring, 0.3)


Matriz de confusión
      0    1
0  962  333
1   74  730

roc auc: 0.916

accuracy: 0.806

precision: 0.687

recall: 0.908

F1: 0.782
